In [1]:
import pandas as pd
import help

X_class, X_family, y_class, y_family = help.loadDataset(5, feature_scailing=True)
print(X_class.shape)
print(X_family.shape)
print(y_class.shape)
print(y_family.shape)

(2000, 12377)
(500, 12377)
(2000,)
(500, 10)


In [2]:
from sklearn.cross_validation import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_class,y_class,test_size=0.2,random_state=0)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

/root/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(1600, 12377)
(400, 12377)
(1600,)
(400,)


In [3]:
def keras_nn_for_class(X_train, X_test, Y_train, Y_test, dropout = 0.5, batch_size = 32):
    from keras.models import Sequential
    from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
    from keras.layers.convolutional import Convolution2D
    
    input_node = X_train.shape[1]
    
    layer = []
    batch_size = 16
    
    train_generator = generator(X_train, Y_train, batch_size=batch_size)
    validation_generator = generator(X_test, Y_test, batch_size=batch_size)
    
    input_shape =(X_train.shape[1],)  # Trimmed image format
    #print("input_shape:", input_shape)
        
    model = Sequential()
    # Preprocess incoming data, centered around zero with small standard deviation 
    model.add(Dense(input_node, activation='relu', input_shape=input_shape , kernel_initializer='normal'))
    layer.append(input_node)
    model.add(Dropout(dropout)) # for preventing overfit
    layer.append("Dropout")
    input_node = input_node//2
    
    
    while(input_node > 1):
        model.add(Dense(input_node, activation='relu',  kernel_initializer='normal'))
        layer.append(input_node)
        model.add(Dropout(dropout)) # for preventing overfit
        layer.append("Dropout")
        input_node = input_node//2
        
        
    model.add(Dense(1, activation='sigmoid', kernel_initializer='normal'))
    layer.append(1)
    #print('layer:', layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #########################################################################
    # train a model

    history_object = model.fit_generator(train_generator, steps_per_epoch=len(X_train)/batch_size,
                        validation_data=validation_generator,
                        validation_steps=len(X_test)/batch_size, epochs=10,verbose=0)
    
    #generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)
    
    loss, score = model.evaluate_generator(validation_generator, steps=len(X_test)/batch_size, max_queue_size=10, workers=1, use_multiprocessing=True)
    return score

def generator(X, y, batch_size=32):
    from sklearn.utils import shuffle
    
    num_samples = len(X)
    
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_X = X[offset:offset+batch_size]
            batch_y = y[offset:offset+batch_size]
        
            yield shuffle(batch_X, batch_y)

In [4]:
keras_nn_for_class(X_train, X_test, Y_train, Y_test)

Using TensorFlow backend.


0.76500000000000001

In [5]:
from sklearn.cross_validation import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_family,y_family,test_size=0.2,random_state=0)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(400, 12377)
(100, 12377)
(400, 10)
(100, 10)


In [6]:
def keras_nn_for_family(X_train, X_test, Y_train, Y_test, dropout = 0.5, batch_size = 32):
    from keras.models import Sequential
    from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
    from keras.layers.convolutional import Convolution2D
    
    input_node = X_train.shape[1]
    layer = []
    
    
    train_generator = generator(X_train, Y_train, batch_size=batch_size)
    validation_generator = generator(X_test, Y_test, batch_size=batch_size)
    
    input_shape =(X_train.shape[1],)  # Trimmed image format
    #print("input_shape:", input_shape)
        
    model = Sequential()
    # Preprocess incoming data, centered around zero with small standard deviation 
    model.add(Dense(input_node, activation='relu', input_shape=input_shape , kernel_initializer='normal'))
    layer.append(input_node)
    model.add(Dropout(dropout)) # for preventing overfit
    layer.append("Dropout")
    input_node = input_node//2
    
    
    while(input_node > 10):
        model.add(Dense(input_node, activation='relu',  kernel_initializer='normal'))
        layer.append(input_node)
        model.add(Dropout(dropout)) # for preventing overfit
        layer.append("Dropout")
        input_node = input_node//2
        
        
    model.add(Dense(10, activation='softmax', kernel_initializer='normal'))
    layer.append(10)
    #print('layer:', layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #########################################################################
    # train a model

    history_object = model.fit_generator(train_generator, steps_per_epoch=len(X_train)/batch_size,
                        validation_data=validation_generator,
                        validation_steps=len(X_test)/batch_size, epochs=10, verbose=0)
    
    loss, score = model.evaluate_generator(validation_generator, steps=len(X_test)/batch_size, max_queue_size=10, workers=1, use_multiprocessing=True)
    return score

def generator(X, y, batch_size=32):
    from sklearn.utils import shuffle
    
    num_samples = len(X)
    
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_X = X[offset:offset+batch_size]
            batch_y = y[offset:offset+batch_size]
        
            yield shuffle(batch_X, batch_y)

In [7]:
keras_nn_for_family(X_train, X_test, Y_train, Y_test)

0.089999999999999997